# Package Import

In [ ]:
import pandas as pd
from glob import glob
from canvasapi import Canvas
import re
import os

# Setup
## Running the script for the first time
1. Generate Canvas token and assign it to `API_KEY`
2. Get Canvas course ID and assign it to `COURSE_ID`
3. Create an assignment group on canvas

## Configurations
- `API_URL`: url of the canvas domain
- `API_KEY`: Canvas key to tutor's account (can be generated in Account -> Settings -> New Access Token)
- `COURSE_ID`: ID of the course (the 5-digit number in the url of the course's canvas page)
- `all_files`: list of the paths to participation file from sli.do (analytics -> export -> Poll results per user)
- `threshold`: Fraction of questions a student can miss and still get the credit
- `group_name`: Name of the assignment group for attendance

In [ ]:
API_URL = "https://canvas.ucsd.edu/"
API_KEY = ...
COURSE_ID = ... 
all_files = ['marina/Polls-per-user-Lecture0_09-24.xlsx']
threshold = 0.25
group_name = "Lecture Attendance"

In [ ]:
# Login to Canvas
canvas = Canvas(API_URL, API_KEY)
course = canvas.get_course(COURSE_ID)
print(f'Successfully logged in to: {course.name}')

# Parse Sli.do result

In [ ]:
### Process Sli.do result
names = []
scores = []
for fname in all_files:
    # Find the name of the assignment Lecture00 or Lec00, modify if needed
    name = re.findall("((?:Lecture|Lec)[0-9]+)",fname)[0]
    names.append(name)
    
    # read file
    raw = pd.read_excel(fname, header = 0, skiprows=[1])
    df = raw.drop(columns = ["User ID", "User Name", "User company", "Total Correct Answers"]).set_index("User Email")
    
    # drop question if less than 2 people answered it 
    df = df.drop(columns = df.columns[df.notna().sum()<=1])
    
    # Calculate score for each student
    scores.append((df.isna().mean(axis = 1) < threshold).rename(name).to_frame())
print(names)

# Get assignment group ID

In [ ]:
### Find assisgnment group
group_id = -1
for group in course.get_assignment_groups():
    if group.name == group_name:
        group_id = group.id
        break
assert group_id != -1, "Manually create the assignemt group first"
group_id

# Get student info from Canvas 
**run this cell only when new student is added to the course**

In [ ]:
### Get student info: id - Name - Email
# student_ls = []
# for stu in course.get_users(enrollment_type=['student'], enrollment_state=['active']):
#     student_ls.append([stu.id, stu.name, stu.get_profile()["primary_email"]])
# students = pd.DataFrame(student_ls, columns = ["id", "name", "email"])
# students.to_csv("students.csv")
# print(f'Number of students: {len(students)}')

In [ ]:
assert os.path.exists("students.csv"), "Run the cell above"
students = pd.read_csv("students.csv", index_col=0)
print(len(students))

# Upload score to Canvas

In [ ]:
# Join tables
joined = students.copy().set_index("email")
for score in scores:
    joined = joined.join(score, how="outer")
joined = joined.fillna(False)
joined.tail()

In [ ]:
# check for duplication
existed_assignment = [a.name for a in course.get_assignments()]

In [ ]:
for col in names:
    if col in existed_assignment:    # skip if assignment already existed
        continue
        
    # create new assignment
    new_assignment = course.create_assignment({ 
        'name': col,
        'notify_of_update': False,
        'points_possible': 1,
        'published': True,
        "assignment_group_id":group_id
    })
    
    # Submit score 
    for email, id  in joined["id"].items():
        if id == False:
            print(f'Student with email {email} not found on canvas')
            continue
        if joined[col][email]:
            new_assignment.get_submission(id).edit(submission={'posted_grade': 1})
        else:
            new_assignment.get_submission(id).edit(submission={'posted_grade': 0})

# Check if the score is updated correctly!!!